# Exercice 7
## Modifier les données en masse avec apply
### Consigne
Le fichier `Data/titles_not_linked.csv` contient une liste de titres.

Dans ce fichier, réaliser les tâches suivantes:
* Créer une table en ne retenant que les entrées avec ISBN
* Extraire le premier ISBN dans une nouvelle colonne
* Vérifier la conformité de l'ISBN (juste la longuer 10 ou 13 et non l'algorithme précis)
* Créer une table avec les ISBN douteux: `Resultat/isbn_douteux.xlsx`

**Exercice avancé:** 
* Prendre l'ensemble des données et rassembler les colonnes ISBN et ISSN en une seule: "Identifiant"
* Supprimer les colonnes "ISBN" et "ISSN"
* Supprimer les lignes avec des valeurs nulles dans les identifiants

In [1]:
# Importer les bibliothèques requises
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Chargement des données
df = pd.read_csv('Data/titles_not_linked.csv')

In [3]:
df.head()

,MMS Id,Linked to NZ,Title,ISBN,ISSN,Linked to CZ
0,990000076740205516,No,"Livres hebdo l'hebdomadaire des livres, éditio...",NaN,0294-0000,No
1,990000624650205516,No,Cahier / Alliance Culturelle Romande,NaN,NaN,No
2,990000816850205516,No,NaN,NaN,NaN,No
3,990000937220205516,No,: carte in parte col.,NaN,NaN,No
4,990000937230205516,No,: carte in parte col.,NaN,NaN,No


### Filtrer la table en ne retenant que les lignes avec ISBN

In [4]:
# Supprime les lignes où la colonne 'ISBN' ne comporte pas de valeur.
# Avec la méthode 'dropna', c'est en général une bonne pratique de faire une copie
# indépendante des données
d_isbn = df.dropna(subset=['ISBN']).copy()

# Suppression de la colonne 'ISSN', le paramètre 'inplace' permet de ne pas
# retourner le nouveau DataFrame, mais de modifier les données directement.
d_isbn.drop('ISSN', axis=1, inplace=True)

In [5]:
d_isbn.head()

,MMS Id,Linked to NZ,Title,ISBN,Linked to CZ
6,990001353040205516,No,NaN,0854980326; 9780854980321,No
7,990001475420205516,No,NaN,0306335026; 9780306335020,No
12,990001876030205516,No,NaN,3764308613; 9783764308612,No
13,990001878950205516,No,NaN,9264215956; 9789264215955,No
15,990001967100205516,No,NaN,2850320099; 9782850320095,No


### Récupérer le premier ISBN

In [6]:
# "split" permet de transformer en liste le contenu en séparent par ";"
# "strip" élimine les espaces résiduels
d_isbn['First ISBN'] = d_isbn['ISBN'].apply(lambda x: x.split(';')[0].strip())

In [7]:
d_isbn['First ISBN']

6          0854980326
7          0306335026
12         3764308613
13         9264215956
15         2850320099
            ...      
7923    9781449308292
7925           ISBN10
7926       2880749093
7927    9788894545708
7928       376436050X
Name: First ISBN, Length: 3278, dtype: object

### Vérifier la longueur de l'ISBN

In [8]:
# Teste la longueur de l'ISBN
d_isbn['Longueur ISBN est normale'] = d_isbn['First ISBN'].apply(lambda x: len(x)==10 or len(x)==13)

In [9]:
# Création de la table avec uniquement les ISBN douteux
# le "~" est une négation
# Avec la méthode "dropna", il est souvent utile de
d_isbn_douteux = d_isbn.loc[~d_isbn['Longueur ISBN est normale']].copy()
d_isbn_douteux.head()

,MMS Id,Linked to NZ,Title,ISBN,Linked to CZ,First ISBN,Longueur ISBN est normale
609,990005875580205516,No,NaN,306170655,No,306170655,False
727,990006148470205516,No,NaN,624694605200,No,624694605200,False
2898,990017430620205516,No,NaN,007036261,No,007036261,False
3931,99116724845005516,No,Le sauvage le Nouvel observateur - écologie,03397262,No,03397262,False
4059,99116725150205516,No,The bearing strength of concrete for strip loa...,https://doi.org/10.1680/macr.1970.22.71.87,No,https://doi.org/10.1680/macr.1970.22.71.87,False


In [10]:
# Export de la table
d_isbn_douteux.to_excel('Resultat/isbn_douteux.xlsx', index=False)

## Exercice avancé

In [17]:
# %%timeit
# Version condensée avec une fonction lambda

# Chargement des données
df = pd.read_csv('Data/titles_not_linked.csv')

df['Identifiant'] = df.apply(lambda row: row['ISBN'] if pd.notnull(row['ISBN']) else row['ISSN'], axis=1)
len(df)

7931

In [18]:
# %%timeit
# Version avec une fonction
def check_identifier_1(row):
    if pd.notnull(row['ISBN']):
        return row['ISBN']
    if pd.notnull(row['ISSN']):
        return row['ISSN']
    return np.nan

# Chargement des données
df = pd.read_csv('Data/titles_not_linked.csv')

df['Identifiant'] = df.apply(check_identifier_1, axis=1)
len(df)

7931

In [19]:
# %%timeit
# Version avec modification de la table par "apply"
def check_identifier_2(row):
    """
    Cette fonction retourne la ligne avec le report de l'ISBN ou ISSN
    dans la colonne "Identifiant".
    """
    if pd.notnull(row['ISBN']):
        row['Identifiant'] = row['ISBN']

    if pd.notnull(row['ISSN']):
        row['Identifiant'] = row['ISSN']
    
    return row

# Chargement des données
df = pd.read_csv('Data/titles_not_linked.csv')
df['Identifiant'] = np.nan
df = df.apply(check_identifier_2, axis=1)
len(df)

7931

In [20]:
df.head()

,MMS Id,Linked to NZ,Title,ISBN,ISSN,Linked to CZ,Identifiant
0,990000076740205516,No,"Livres hebdo l'hebdomadaire des livres, éditio...",NaN,0294-0000,No,0294-0000
1,990000624650205516,No,Cahier / Alliance Culturelle Romande,NaN,NaN,No,NaN
2,990000816850205516,No,NaN,NaN,NaN,No,NaN
3,990000937220205516,No,: carte in parte col.,NaN,NaN,No,NaN
4,990000937230205516,No,: carte in parte col.,NaN,NaN,No,NaN


In [21]:
# Suppression des colonnes "ISSN" et "ISBN"
df.drop(['ISSN', 'ISBN'], axis=1, inplace=True)

In [22]:
# Suppression des valeurs nulles
df = df.dropna(subset=['Identifiant']).copy()

In [23]:
df.head()

,MMS Id,Linked to NZ,Title,Linked to CZ,Identifiant
0,990000076740205516,No,"Livres hebdo l'hebdomadaire des livres, éditio...",No,0294-0000
6,990001353040205516,No,NaN,No,0854980326; 9780854980321
7,990001475420205516,No,NaN,No,0306335026; 9780306335020
12,990001876030205516,No,NaN,No,3764308613; 9783764308612
13,990001878950205516,No,NaN,No,9264215956; 9789264215955
